In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
from ln_utils import *
from transaction_simulator import *

In [ ]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

### WHAT TO DO WITH INF-s???

experiment_id = "2019-05-30_18:55:02_100000sat_k10000"
snapshots = range(7)
simulation_dir = "/mnt/idms/fberes/data/bitcoin_ln_research/simulations/"

experiment_files = get_experiment_files(experiment_id, snapshots, simulation_dir)

all_infos = []
for snap_id in snapshots:
    info, df = aggregate_samples(experiment_files, snap_id)
    all_infos.append(info)
    print(snap_id)

all_infos[0]

### Average correlation (over time) between cost changes and other metric

methods = ["spearman","wkendall","kendall"]
fig, axis = plt.subplots(len(methods), 1, figsize=(10,15))
for i, m in enumerate(methods):
    mx = corr_mx(all_infos[0].drop("removed_node", axis=1), method=m)
    for j in range(1,len(all_infos)):
        mx = np.add(mx, corr_mx(all_infos[j].drop("removed_node", axis=1), method=m))
    axis[i].set_title(m)
    sns.heatmap(mx / len(all_infos), annot=True, ax=axis[i], vmin=-0.3, vmax=1.0)

# Node analysis with 1ML background

from scrapy.crawler import CrawlerProcess
from scraping import *

### Average metrics of nodes

mean_all_info = pd.concat(all_infos, sort=True).groupby("removed_node").mean()
mean_all_info = mean_all_info.sort_values("cost_diff", ascending=False)

def show_df_with_alias(df):
    nodes = list(df.index)
    process = CrawlerProcess()
    process.crawl(LNNodeParser(nodes))
    process.start()
    print()

show_df_with_alias(mean_all_info.head(10))

mean_all_info.head(10)

mean_all_info.tail(10)

# 1. Load data

In [ ]:
def load_data(experiment_files, snapshots, file_postfix):
    parts = []
    for snap_id in snapshots:
        data_files = []
        for f in experiment_files[snap_id]:
            sample_id = f.split(".")[-1]
            if file_postfix == "router_incomes":
                df = pd.read_csv("%s/%s.csv" % (f, file_postfix), header=None)
                df.columns = ["node","income"]
            else:
                df = pd.read_csv("%s/%s.csv" % (f, file_postfix))
            df["sample"] = int(sample_id)
            data_files.append(df)
        tmp_df = pd.concat(data_files, axis=0, sort=False)
        tmp_df["snapshot_id"] = snap_id
        parts.append(tmp_df)
        print(snap_id)
    return parts

In [ ]:
#experiment_id = "2019-07-04_11:12:08_200000sat_k30000"
#experiment_id = "2019-07-05_14:39:09_200000sat_k30000"
#experiment_id = "2019-07-05_14:39:09_500000sat_k30000"
#experiment_id = "2019-08-06_13:42:00_200000sat_k30000_aNone"
#experiment_id = "2019-08-09_14:31:08_200000sat_k30000_aNone_dropTrue"
#experiment_id = "2019-08-09_14:31:08_127807sat_k50000_aNone_dropTrue"
experiment_id = "2019-09-01_18:07:51_200000sat_k30000_aNone_dropTrue"
snapshots = range(7)
simulation_dir = "/mnt/idms/fberes/data/bitcoin_ln_research/simulations/"

In [ ]:
experiment_folders = get_experiment_files(experiment_id, snapshots, simulation_dir)

In [ ]:
experiment_folders[0]

In [ ]:
pricing_pol = load_data(experiment_folders, snapshots, "opt_fees")

In [ ]:
source_fee = load_data(experiment_folders, snapshots, "source_fees")

In [ ]:
router_income = load_data(experiment_folders, snapshots, "router_incomes")

## Router incomes

In [ ]:
x = snapshots
y = [router_income[i]["income"].mean() for i in snapshots]
plt.plot(x,y, "bo")

In [ ]:
sns.jointplot(data=pd.concat(router_income), x="snapshot_id", y="income")

In [ ]:
all_router_incomes = pd.concat(router_income)

In [ ]:
all_router_incomes = all_router_incomes.groupby("node")["income"].mean().sort_values(ascending=False)

### Top 10 router income for 30K transactions (200K SAT amount):

- zigzag.io 212K (?)
- ln1.satoshilabs.com 71K
- yalls.org 50K
- BlueWallet 22K (Node: 03021c5f5f57322740e4)
- LNBIG.com [lnd-26]: 21K
- LNBIG.com [lnd-37]: 20K
- LNBIG.com [lnd-38]: 16K
- LNBIG.com [lnd-33]: 15K
- LNBIG.com [lnd-25]: 12K
- LNBIG.com [lnd-17]: 12K
- LNBIG.com [lnd-03]: 12K
- LNBIG.com [lnd-01]: 11K
- LNBIG.com [lnd-09]: 11K
- LNBIG.com [lnd-21]: 10K
- LNBIG.com [lnd-05]: 9K
- LNBIG.com [lnd-34]: 8K
- OpenNode: 8233.32 (17. csúcs)
- LNBIG.com [lnd-31]: 7800
- LNBIG.com [lnd-07]: 7700
- LNBIG.com [lnd-06]: 7729
- bitBruno: 7465 (21. csúcs)
- LNBIG.com [lnd-41]: 7064
- LNBIG.com [lnd-28/old-lnd-22]: 6914
- LNBIG.com [lnd-02]: 6870
- tippin.me: 6808 (25. csúcs)
- LNBIG.com [lnd-32]: 6393
- LNBIG.com [lnd-10]: 5889
- LNBIG.com [lnd-42]: 5548
- btc.lnetwork.tokyo: 3793
- KZIN: 3167

#### further in the list:
- ACINQ: 2660
- 1ML.com node alpha: 1457


#### LNBIG

- **226K SAT income for LNBIG (from its most profitable 20 nodes) - 21,56 USD** - according to him daily 20 USD a month (5-10K SAT per day / 200-300 comission per day)

lnbig_indices = list(range(4,16)) + list(range(17,20)) + [21,22,23] + [25,26,27]
len(lnbig_indices)

incomes_arr = list(all_router_incomes)[:30]
#lnbig_incomes = incomes_arr[lnbig_indices]

lnbig_total = 0.0
for idx in lnbig_indices[:20]:
    lnbig_total += incomes_arr[idx]

lnbig_total

all_router_incomes.head(50)

## Source fees

In [ ]:
x = snapshots
y = [source_fee[i]["mean_fee"].mean() for i in snapshots]
plt.plot(x,y, "bo")

In [ ]:
sns.jointplot(data=pd.concat(source_fee), x="snapshot_id", y="mean_fee")

## Optimal pricing

In [ ]:
x = snapshots
y = [pricing_pol[i]["opt_delta"].mean() for i in snapshots]
plt.plot(x,y, "bo")

In [ ]:
x = snapshots
y = [pricing_pol[i]["opt_traffic"].mean() for i in snapshots]
plt.plot(x,y, "bo")

In [ ]:
x = snapshots
y = [pricing_pol[i]["origi_income"].mean() for i in snapshots]
plt.plot(x,y, "bo")

In [ ]:
x = snapshots
y = [pricing_pol[i]["opt_income"].mean() for i in snapshots]
plt.plot(x,y, "bo")

### Calculate income revenue

In [ ]:
for df in pricing_pol:
    df["income_diff"] = df["opt_income"] - df["origi_income"]

# 2. Simulation stability

In [ ]:
corrs = ["spearman","kendall","wkendall"]

In [ ]:
def avg_cross_corr(df, snapshot_id, col, methods=corrs, key_col="node"):
    snap = df[snapshot_id]
    sample_num = snap["sample"].max()+1
    cols = [key_col, col]
    merged = snap[snap["sample"]==0][cols].rename({col:col+"_0"}, axis=1)
    for i in range(1,sample_num):
        s = snap[snap["sample"]==i][cols].rename({col:col+"_%i" % i}, axis=1)
        merged = merged.merge(s, on=key_col, how="outer").fillna(0.0)
    merged.drop("node", inplace=True, axis=1)
    res = {}
    for method in methods:
        cnt = corr_mx(merged, method=method).sum().sum() - sample_num 
        denom = sample_num**2-sample_num
        res[method] = cnt / denom
    return res

In [ ]:
origi_inc = pd.DataFrame([avg_cross_corr(pricing_pol, snap_id, "origi_income") for snap_id in snapshots])
opt_inc = pd.DataFrame([avg_cross_corr(pricing_pol, snap_id, "opt_income") for snap_id in snapshots])
income_diff = pd.DataFrame([avg_cross_corr(pricing_pol, snap_id, "income_diff") for snap_id in snapshots])
opt_delta = pd.DataFrame([avg_cross_corr(pricing_pol, snap_id, "opt_delta") for snap_id in snapshots])

In [ ]:
colors = ['b','g','r','m']
fig, axis = plt.subplots(1,3,figsize=(15,4))
x = snapshots
# incomes
axis[0].set_title("INCOME mean cross correlations")
for i, c in enumerate(corrs):
    axis[0].plot(x,origi_inc[c], colors[i]+'-', label=c+"_orig")
    axis[0].plot(x,opt_inc[c], colors[i]+'--', label=c+"_opt")
axis[0].set_xlabel("snapshot")
axis[0].legend()
# income_diff
axis[1].set_title("INCOME_DIFF mean cross correlations")
for i, c in enumerate(corrs):
    axis[1].plot(x,income_diff[c], colors[i]+'-', label=c)
axis[1].set_xlabel("snapshot")
axis[1].legend()
# opt_delta
axis[2].set_title("OPT_DELTA mean cross correlations")
for i, c in enumerate(corrs):
    axis[2].plot(x,opt_delta[c], colors[i]+'-', label=c)
axis[2].set_xlabel("snapshot")
axis[2].legend()
plt.show()

#### Incomes observations

- taking mean cross correlation of user incomes (original and optimal)
- weighted kendall-tau gives decreased to 0.90 from 0.95 (after including capacity maintenance)
- in case of unweighted spearman and kendall the optimal income correlates better across samples 

#### income_diff observations¶

- taking mean cross correlation of income difference (optimal income - original income)
- weighted kendall-tau decreased to 0.80 from 0.93 (after including capacity maintenance)

#### opt_delta observations¶

- taking mean cross correlation of optimal base fee change
- weighted kendall-tau decreased to 0.70 from 0.90 (after including capacity maintenance)

**CONCLUSION: our base_fee optimization procedure is less efficient (meaningful) after the implementation of capacity maintenance!!!**

# 3. Income revenue analysis

In [ ]:
def cut_into_categories(df):
    #categories = [df[:10], df[10:30], df[30:50], df[50:100]]
    #categories = [df[:20], df[20:50], df[50:100], df[100:200]]
    #categories = [df[:10], df[10:20], df[20:30], df[30:40], df[40:50], df[50:100], df[100:500]]
    categories = [df[:10], df[10:30], df[30:50], df[50:100], df[100:500]]
    #categories = [df[:50], df[50:100], df[100:500]]#, df[200:500]]
    return categories

def merge_categories(categories):
    for i, cat in enumerate(categories):
        cat["category"] = i
    return pd.concat(categories)
    
def show_stats(snap_id, col="income_diff"):
    mean = pricing_pol[snap_id].groupby("node").mean().sort_values("origi_income", ascending=False)
    categories = cut_into_categories(mean)
    return [float(cat[[col]].mean()) for cat in categories]

In [ ]:
x = range(5)

### Visualize each snapshot

df = pd.DataFrame([show_stats(i, "income_diff") for i in snapshots])

plt.Figure(figsize=(10,10))
for i in snapshots:
    plt.plot(x, show_stats(i, "income_diff"), label=i)
plt.yscale("log")
plt.legend()

plt.Figure(figsize=(10,10))
for i in snapshots:
    plt.plot(x, show_stats(i, "opt_traffic"), label=i)
plt.legend()

### Average stats (over all snapshots and samples)

average_stats = pricing_pol[0].groupby("node").mean().sort_values("origi_income", ascending=False)[["income_diff","origi_income","opt_income","opt_traffic","opt_delta"]]

In [ ]:
all_records = pd.concat([pricing_pol[i] for i in snapshots])

In [ ]:
average_stats = all_records.groupby("node").mean().sort_values("origi_income", ascending=False)[["income_diff","origi_income","opt_income","opt_traffic","opt_delta"]]

In [ ]:
average_stats_cats = cut_into_categories(average_stats)
mean_diff = [float(cat[["income_diff"]].mean()) for cat in average_stats_cats]
mean_traffic = [float(cat[["opt_traffic"]].mean()) for cat in average_stats_cats]
mean_delta = [float(cat[["opt_delta"]].mean()) for cat in average_stats_cats]

In [ ]:
fig, axis = plt.subplots(1,3,figsize=(15,4))
axis[0].plot(x, mean_diff, 'rx')
axis[0].set_xlabel("income category")
axis[0].set_ylabel("mean income_diff")
axis[1].plot(x, mean_delta, 'g*')
axis[1].set_xlabel("income category")
axis[1].set_ylabel("mean opt_delta")
axis[2].plot(x, mean_traffic, 'bo')
axis[2].set_xlabel("income category")
axis[2].set_ylabel("mean opt_traffic")
plt.show()

#### Mean original income for the selected categories

In [ ]:
[float(cat[["origi_income"]].mean()) for cat in average_stats_cats]

### PROBLEM with outliers...

In [ ]:
merged = merge_categories(average_stats_cats)

In [ ]:
sns.boxplot(data=merged, x="category", y="income_diff", whis=2)

average_stats.head(10)

### Nodes above 20K SAT (2.5 USD) income_diff (with alpha=2.0 source distribution)

#### 2019-07-04_11:12:08_200000sat_k30000 (ordered by original income)

1. yalls.org - Read and write articles, with Lightning Network micropayments. (195,925.46 USD)
2. ACINQ - ACINQ is one of the leading companies working on Bitcoin scalability. (507,211.38 USD)
3. 1ML.com node ALPHA (89,453.40 USD)
4. LightningPowerUsers.com (313,406.05 USD)
5. LightningTo.Me - Helping to resolve routing and capacity issues (165,531.35 USD)
6. tomjodh.nl - Buy photos with satoshis (8,709.95 USD) - **TODO RECLASSIFY AS merchant!!!**
7. ? - ? (848.05 USD)

#### 2019-07-05_14:39:09_200000sat_k30000 (ordered by original income)

1. ACINQ - ACINQ is one of the leading companies working on Bitcoin scalability. (507,211.38 USD, 783 channel)
2. 1ML.com node ALPHA (89,453.40 USD, 884 channel)
3. LightningPowerUsers.com (313,406.05 USD, 1,266 channel)
4. LightningTo.Me - Helping to resolve routing and capacity issues (165,531.35 USD, 700 channel)
5. tomjodh.nl - Buy photos with satoshis (8,709.95 USD, 47 channel)
6. ? - ? (859.23 USD, 4 channel)
7. nybemotion - ? (318.35 USD, 3 channel)

#### Additional nodes in 2019-07-05_14:39:09_500000sat_k30000

4. Bitrefill.com - Buy Gift Cards with Bitcoin for 30+ supported services (310,369.63 USD, 305 channel) - **TODO RECLASSIFY AS merchant!!!**
6. tady je slushovo: ? (48,872.23 USD, 336 channel)

### Nodes above 20K SAT (2.5 USD) income_diff (with alpha=None source distribution)

#### 2019-08-06_13:42:00_200000sat_k30000_aNone (ordered by original income)

1. yalls.org
2. ACINQ
3. LightningPowerUsers.com
4. tomjodh.nl
5. Node: 0297b368360113e29032 ???
6. nybemotion

#### 2019-08-09_14:31:08_200000sat_k30000_aNone_dropTrue

1. yalls.org
2. ln1.satoshilabs.com (306,117.70 USD - 824 channels) **(NEW)**
3. LightningPowerUsers.com
4. nybemotion (493.61 USD, 6 channel) ???
5. Amazing_Droopy_3 (113.73 USD, 1 channel?) ???

#### 2019-09-01_18:07:51_200000sat_k30000_aNone_dropTrue (WITH capacity maintenance)

1. LightningPowerUsers.com (237,653 USD, 1261 channels)
2. 1ML.com node ALPHA (64,630 USD, 889 channels)
3. CoinGate (147,272 USD, 462 channels) - CoinGate is the first large cryptocurrency payment gateway to introduce Lightning Network payments to hundreds of merchants with no technical knowledge required
4. lndhub.ru (19,066 USD, 248 channels) - increase connectivity
5. BeerselLightningNode (13,246 USD, 84 channels)
6. BitKassa LN34961 (13,631 USD, 47) - some kind of payment API and BTC tech company
7. Serving Beer (2,107 USD, 13 channels) - is this really a merchant???

In [ ]:
average_stats[average_stats["income_diff"]>20000]